# vLLM 离线推理示例

这个 notebook 展示如何使用 vLLM 离线推理（无需启动服务器）来运行 benchmark 评估。

## 优势
- ✅ 无需启动和管理 vLLM 服务器
- ✅ 自动批处理，更高效的数据并行
- ✅ 更低的延迟（无网络开销）
- ✅ 完全兼容 DSPy 和 GEPA

## 对比：服务器模式 vs 离线模式

| 特性 | 服务器模式 | 离线模式 |
|------|------------|----------|
| 启动方式 | 需要先运行 `bash start_vllm.sh` | 直接在代码中加载 |
| 网络开销 | 有（HTTP 请求） | 无 |
| 并行推理 | 需手动管理并发 | 自动批处理 |
| 资源占用 | 服务器 + 客户端 | 仅推理引擎 |
| 适用场景 | 多客户端、生产环境 | 单机批量推理、实验 |

## 步骤 1: 初始化 vLLM 离线推理

这会直接加载模型到 GPU，无需服务器。

In [ ]:
import dspy
from vllm_dspy_adapter import vLLMOffline

# 初始化 vLLM 离线推理引擎
# 注意：第一次运行会加载模型，需要几分钟
lm = vLLMOffline(
    model="/home/yuhan/model_zoo/Qwen3-8B",  # 模型路径
    temperature=0.6,
    max_tokens=15000,
    top_p=0.95,
    gpu_memory_utilization=0.9,  # GPU 显存利用率
    tensor_parallel_size=1,      # 单GPU设为1，多GPU可设为GPU数量
)

# 配置 DSPy 使用这个语言模型
dspy.configure(lm=lm)

print("\n✓ vLLM 离线推理引擎已就绪！")

## 步骤 2: 测试基础推理

验证模型能够正常工作。

In [ ]:
# 单个推理测试
result = lm("1+1等于几？请只回答数字。")
print(f"问题: 1+1等于几？")
print(f"回答: {result[0]}")

## 步骤 3: 批量推理（数据并行）

vLLM 的核心优势：自动批处理多个请求。

In [ ]:
import time

# 准备多个问题
questions = [
    "1+1=?",
    "什么是深度学习？",
    "解释一下transformer架构。",
    "Python中如何定义函数？",
    "什么是注意力机制？",
]

# 批量推理
start_time = time.time()
results = lm.batch_generate(questions, max_tokens=200)
end_time = time.time()

print(f"批量推理 {len(questions)} 个问题，耗时: {end_time - start_time:.2f} 秒")
print(f"平均每个问题: {(end_time - start_time) / len(questions):.2f} 秒\n")

# 显示结果
for i, (q, a) in enumerate(zip(questions, results), 1):
    print(f"[{i}] 问题: {q}")
    print(f"    回答: {a[:100]}...\n")

## 步骤 4: 加载 Benchmark

使用 AIME 数学基准测试（与原 notebook 相同）。

In [ ]:
from gepa_artifact.benchmarks.AIME import benchmark as aime_metas

# 加载 benchmark
cur_meta = aime_metas
bench = cur_meta[0].benchmark()

print(f"训练集大小: {len(bench.train_set)}")
print(f"验证集大小: {len(bench.val_set)}")
print(f"测试集大小: {len(bench.test_set)}")

# 查看一个样本
print("\n示例问题:")
print(bench.test_set[0])

## 步骤 5: 加载 Program

这是用于解决问题的 DSPy 程序。

In [ ]:
# 加载预定义的程序
program = cur_meta[0].program[0]
print("Program 结构:")
print(program)

## 步骤 6: 评估 Base Program

使用多线程并行评估（num_threads=80 利用 vLLM 的批处理能力）。

In [ ]:
# 创建评估器
evaluate = dspy.Evaluate(
    devset=bench.test_set,
    metric=cur_meta[0].metric,
    num_threads=80,  # 多线程评估，vLLM 会自动批处理
    display_table=True,
    display_progress=True,
    max_errors=100 * len(bench.test_set)
)

# 开始评估
print("开始评估 base program...")
base_score = evaluate(program)
print(f"\n✓ Base Program 得分: {base_score}")

## 步骤 7: 使用 GEPA 优化（可选）

使用 GEPA 优化器来改进程序性能。

In [ ]:
import os
import time
from gepa_artifact.gepa.gepa import GEPA
from gepa_artifact.utils.capture_stream_logger import Logger

# 创建运行目录
runs_dir = os.path.join(os.getcwd(), "runs", time.strftime("%Y-%m-%d_%H-%M-%S"))
os.makedirs(runs_dir, exist_ok=True)

gepa_logger = Logger(os.path.join(runs_dir, "run_log.txt"))

# 定义 feedback 函数
if cur_meta[0].feedback_fn_maps is None or cur_meta[0].feedback_fn_maps[0] is None:
    def feedback_func(predictor_output, predictor_inputs, module_inputs, module_outputs, captured_trace):
        pred = cur_meta[0].metric_with_feedback(module_inputs, module_outputs, None)
        return {
            "feedback_score": pred.score,
            "feedback_text": pred.feedback,
        }
    feedback_fn_map = {k: feedback_func for k, v in program.named_predictors()}
else:
    feedback_fn_map = cur_meta[0].feedback_fn_maps[0]

# 创建 GEPA 优化器
optimizer = GEPA(
    named_predictor_to_feedback_fn_map=feedback_fn_map,
    knowledgebase_qe=None,
    metric=cur_meta[0].metric,
    run_linearized_gepa=False,
    use_merge=True,
    set_for_merge_minibatch='val',
    track_scores_on='val',
    max_metric_calls=700,
    run_dir=runs_dir,
    logger=gepa_logger,
    num_threads=40
)

print("GEPA 优化器已创建")

In [ ]:
# 运行优化
print("开始 GEPA 优化...")
optimized_program = optimizer.compile(
    cur_meta[0].program[0],
    trainset=bench.train_set,
    valset=bench.val_set[:len(bench.val_set)//2],
)
print("\n✓ 优化完成！")

## 步骤 8: 评估优化后的 Program

In [ ]:
# 评估优化后的程序
print("评估优化后的 program...")
optimized_score = evaluate(optimized_program)

print(f"\n" + "="*60)
print(f"Base Program 得分: {base_score}")
print(f"优化后 Program 得分: {optimized_score}")
print(f"提升: {optimized_score - base_score:.2%}")
print("="*60)

## 步骤 9: 查看优化后的 Prompt

In [ ]:
# 打印 GEPA 发现的优化指令
for name, pred in optimized_program.named_predictors():
    print("="*60)
    print(f"Predictor: {name}")
    print("="*60)
    print("Optimized Instructions:")
    print(pred.signature.instructions)
    print("\n")

## 总结

### 使用 vLLM 离线推理的关键点：

1. **无需服务器**：直接在代码中加载模型
2. **自动批处理**：vLLM 自动优化批量推理
3. **多线程支持**：`dspy.Evaluate` 的 `num_threads` 参数配合 vLLM 的批处理实现高效并行
4. **完全兼容**：与现有的 DSPy 和 GEPA 代码无缝集成

### 性能对比（参考）：

- **服务器模式**：每个请求需要 HTTP 开销，适合多客户端场景
- **离线模式**：无网络开销，批处理优化，适合单机大规模评估

### 何时使用离线模式：

✅ 运行 benchmark 评估  
✅ 批量处理大量样本  
✅ GEPA 优化器训练  
✅ 单机实验和研究  

### 何时使用服务器模式：

✅ 多个客户端同时使用  
✅ 生产环境部署  
✅ 需要 API 接口  
✅ 跨机器访问  